# Lab 1: Create an Azure Function

In this lab, you'll create an Azure Function and call it from Python.

## Prerequisites

### 1. Environment Setup
Follow the [Quick Start Guide](../docs/quickstart.md) to set up your Python environment.

### 2. Create an Azure Function App

1. Go to [Azure Portal](https://portal.azure.com)
2. Click **Create a resource** → Search **Function App** → **Create**
3. Fill in:
   - **Subscription**: Your subscription
   - **Resource Group**: Create new or use existing
   - **Function App name**: `msai-functions-<your-name>` (must be unique)
   - **Runtime stack**: Python 3.11
   - **Region**: East US (or closest to you)
4. Click **Review + Create** → **Create**

### 3. Create a Function

1. Go to your Function App → **Functions** → **Create**
2. Select **HTTP trigger**
3. Name it: `analyze_data`
4. Authorization level: **Function** (requires a key)
5. Click **Create**

### 4. Edit the Function Code

1. Click on your function → **Code + Test**
2. Replace the code with:

```python
import azure.functions as func
import json

def main(req: func.HttpRequest) -> func.HttpResponse:
    try:
        data = req.get_json()
        values = data.get('values', [])
        
        result = {
            'count': len(values),
            'sum': sum(values),
            'mean': sum(values) / len(values) if values else 0,
            'min': min(values) if values else None,
            'max': max(values) if values else None
        }
        
        return func.HttpResponse(
            json.dumps(result),
            mimetype='application/json',
            status_code=200
        )
    except Exception as e:
        return func.HttpResponse(
            json.dumps({'error': str(e)}),
            mimetype='application/json',
            status_code=400
        )
```

3. Click **Save**

### 5. Get Your Function URL and Key

1. Click **Get Function URL** → Copy the URL
2. Go to **Function Keys** → Copy the `default` key

---

## Step 1: Setup

In [ ]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

# Your Azure Function details
FUNCTION_URL = os.getenv(
    "AZURE_FUNCTION_URL", "https://your-function-app.azurewebsites.net/api/analyze_data"
)
FUNCTION_KEY = os.getenv("AZURE_FUNCTION_KEY", "your-function-key-here")

print(f"Function URL: {FUNCTION_URL}")

## Step 2: Call Your Azure Function

Send data to your function and get the analysis results.

In [ ]:
# Data to analyze
payload = {"values": [10, 20, 30, 40, 50]}

# Call the Azure Function
response = requests.post(FUNCTION_URL, json=payload, headers={"x-functions-key": FUNCTION_KEY})

if response.status_code == 200:
    result = response.json()
    print("✅ Function called successfully!")
    print(f"   Count: {result['count']}")
    print(f"   Sum: {result['sum']}")
    print(f"   Mean: {result['mean']}")
    print(f"   Min: {result['min']}")
    print(f"   Max: {result['max']}")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

## Step 3: Create a Reusable Function

Wrap the HTTP call in a Python function you can use anywhere.

In [ ]:
def analyze_data(values: list) -> dict:
    """
    Call the Azure Function to analyze a list of numbers.

    Args:
        values: List of numbers to analyze

    Returns:
        Dictionary with count, sum, mean, min, max
    """
    response = requests.post(
        FUNCTION_URL, json={"values": values}, headers={"x-functions-key": FUNCTION_KEY}, timeout=30
    )
    response.raise_for_status()
    return response.json()


# Test it
result = analyze_data([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(f"Analysis result: {result}")

## 🎯 Copilot Tips

### GitHub Copilot Chat Commands
- `/explain` - Get explanation of Azure Functions code
- `/fix` - Fix errors in function calls
- `/tests` - Generate tests for your functions
- `/doc` - Generate docstrings

### Useful Prompts
- "Create a new Azure Function that processes JSON data"
- "Add retry logic to this function call"
- "Generate unit tests for the AzureFunctionsClient"
- "Add error handling for network failures"

## 📚 Additional Resources

- [Azure Functions Documentation](https://learn.microsoft.com/en-us/azure/azure-functions/)
- [Azure AI Foundry](https://azure.microsoft.com/en-us/products/ai-studio/)
- [Python Async/Await Guide](https://docs.python.org/3/library/asyncio.html)

## ✅ Summary

In this lab, you learned:
- ✅ How to configure Azure Functions clients
- ✅ Synchronous and asynchronous function invocation
- ✅ Using pre-built function tools
- ✅ Error handling best practices
- ✅ Integration with AI agents

**Next**: Proceed to Lab 2 to learn about Logic Apps integration!